# Initialization

* implement unifyGNN+Contrastive
* try different noisy and augmentation
* try against with/without unnoticeable 

In [17]:
#!/usr/bin/env python
# coding: utf-8

# In[1]: 

import imp
import time
import argparse
import numpy as np
import torch

from torch_geometric.datasets import Planetoid,Reddit2,Flickr,PPI


# from torch_geometric.loader import DataLoader
# from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated
import scipy.sparse as sp

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--debug', action='store_true',
        default=True, help='debug mode')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=10, help='Random seed.')
parser.add_argument('--model', type=str, default='GCN', help='model',
                    choices=['GCN','GAT','GraphSage','GIN'])
parser.add_argument('--dataset', type=str, default='Cora', 
                    help='Dataset',
                    choices=['Cora','Citeseer','Pubmed','PPI','Flickr','ogbn-arxiv','Reddit','Reddit2','Yelp'])
parser.add_argument('--train_lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=32,
                    help='Number of hidden units.')
parser.add_argument('--num_hidden', type=int, default=128,
                    help='Number of hidden units.')
parser.add_argument('--num_proj_hidden', type=int, default=128,
                    help='Number of hidden units in MLP.')
parser.add_argument('--thrd', type=float, default=0.5)
parser.add_argument('--target_class', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--epochs', type=int,  default=200, help='Number of epochs to train benign and backdoor model.')
parser.add_argument('--trojan_epochs', type=int,  default=400, help='Number of epochs to train trigger generator.')
parser.add_argument('--inner', type=int,  default=1, help='Number of inner')
parser.add_argument('--temperature', type=float,  default=0.5, help='Temperature')
# backdoor setting
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--trigger_size', type=int, default=3,
                    help='tirgger_size')
parser.add_argument('--use_vs_number', action='store_true', default=True,
                    help="if use detailed number to decide Vs")
parser.add_argument('--vs_ratio', type=float, default=0,
                    help="ratio of poisoning nodes relative to the full graph")
parser.add_argument('--vs_number', type=int, default=10,
                    help="number of poisoning nodes relative to the full graph")
# defense setting
parser.add_argument('--defense_mode', type=str, default="none",
                    choices=['prune', 'isolate', 'none'],
                    help="Mode of defense")
parser.add_argument('--prune_thr', type=float, default=0.2,
                    help="Threshold of prunning edges")
parser.add_argument('--target_loss_weight', type=float, default=1,
                    help="Weight of optimize outter trigger generator")
parser.add_argument('--homo_loss_weight', type=float, default=100,
                    help="Weight of optimize similarity loss")
parser.add_argument('--homo_boost_thrd', type=float, default=0.5,
                    help="Threshold of increase similarity")
# attack setting
parser.add_argument('--dis_weight', type=float, default=1,
                    help="Weight of cluster distance")
parser.add_argument('--selection_method', type=str, default='none',
                    choices=['loss','conf','cluster','none','cluster_degree'],
                    help='Method to select idx_attach for training trojan model (none means randomly select)')
parser.add_argument('--test_model', type=str, default='GCN',
                    choices=['GCN','GAT','GraphSage','GIN'],
                    help='Model used to attack')
parser.add_argument('--evaluate_mode', type=str, default='1by1',
                    choices=['overall','1by1'],
                    help='Model used to attack')
# GPU setting
parser.add_argument('--device_id', type=int, default=2,
                    help="Threshold of prunning edges")
# GRACE setting
parser.add_argument('--config', type=str, default="config.yaml")
## Contrasitve setting
parser.add_argument('--cl_lr', type=float, default=0.001)
# parser.add_argument('--cl_num_hidden', type=int, default=64)
parser.add_argument('--cl_num_proj_hidden', type=int, default=128)
parser.add_argument('--cl_num_layers', type=int, default=2)
parser.add_argument('--cl_activation', type=str, default='relu')
parser.add_argument('--cl_base_model', type=str, default='GCNConv')
parser.add_argument('--cont_weight', type=float, default=3)
parser.add_argument('--add_edge_rate_1', type=float, default=0)
parser.add_argument('--add_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_edge_rate_1', type=float, default=0.2)
parser.add_argument('--drop_edge_rate_2', type=float, default=0.5)
parser.add_argument('--drop_feat_rate_1', type=float, default=0.)
parser.add_argument('--drop_feat_rate_2', type=float, default=0)
parser.add_argument('--tau', type=float, default=0.1)
parser.add_argument('--cl_num_epochs', type=int, default=700)
parser.add_argument('--cl_weight_decay', type=float, default=1e-5)
parser.add_argument('--cont_batch_size', type=int, default=0)
parser.add_argument('--clf_weight', type=float, default=1)
parser.add_argument('--inv_weight', type=float, default=1)
parser.add_argument('--select_thrh', type=float, default=0.8)


# args = parser.parse_args()
args = parser.parse_known_args()[0]
args.cuda =  not args.no_cuda and torch.cuda.is_available()
device = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id))

np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# torch.cuda.manual_seed(args.seed)
print(args)
#%%
from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])

if(args.dataset == 'Cora' or args.dataset == 'Citeseer' or args.dataset == 'Pubmed'):
    dataset = Planetoid(root='./data/', \
                        name=args.dataset,\
                        transform=transform)
elif(args.dataset == 'Flickr'):
    dataset = Flickr(root='./data/Flickr/', \
                    transform=transform)
elif(args.dataset == 'Reddit2'):
    dataset = Reddit2(root='./data/Reddit2/', \
                    transform=transform)
elif(args.dataset == 'ogbn-arxiv'):
    from ogb.nodeproppred import PygNodePropPredDataset
    # Download and process data at './dataset/ogbg_molhiv/'
    dataset = PygNodePropPredDataset(name = 'ogbn-arxiv', root='./data/')
    split_idx = dataset.get_idx_split() 

data = dataset[0].to(device)

if(args.dataset == 'ogbn-arxiv'):
    nNode = data.x.shape[0]
    setattr(data,'train_mask',torch.zeros(nNode, dtype=torch.bool).to(device))
    # dataset[0].train_mask = torch.zeros(nEdge, dtype=torch.bool).to(device)
    data.val_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.test_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.y = data.y.squeeze(1)
# we build our own train test split 
#%% 
from utils import get_split
# data, idx_train, idx_val, idx_clean_test, idx_atk = get_split(args,data,device)
idx_train = data.train_mask.nonzero().flatten()
idx_val = data.val_mask.nonzero().flatten()
idx_test = data.test_mask.nonzero().flatten()
idx_clean_test = idx_test[:int(len(idx_test)/2)]
idx_atk = idx_test[int(len(idx_test)/2):]

from torch_geometric.utils import to_undirected
from utils import subgraph
data.edge_index = to_undirected(data.edge_index)
train_edge_index,_, edge_mask = subgraph(torch.bitwise_not(data.test_mask),data.edge_index,relabel_nodes=False)
mask_edge_index = data.edge_index[:,torch.bitwise_not(edge_mask)]
# filter out the unlabeled nodes except from training nodes and testing nodes, nonzero() is to get index, flatten is to get 1-d tensor
unlabeled_idx = (torch.bitwise_not(data.test_mask)&torch.bitwise_not(data.train_mask)).nonzero().flatten()

Namespace(add_edge_rate_1=0, add_edge_rate_2=0, cl_activation='relu', cl_base_model='GCNConv', cl_lr=0.001, cl_num_epochs=700, cl_num_layers=2, cl_num_proj_hidden=128, cl_weight_decay=1e-05, clf_weight=1, config='config.yaml', cont_batch_size=0, cont_weight=3, cuda=True, dataset='Cora', debug=True, defense_mode='none', device_id=2, dis_weight=1, drop_edge_rate_1=0.2, drop_edge_rate_2=0.5, drop_feat_rate_1=0.0, drop_feat_rate_2=0, dropout=0.5, epochs=200, evaluate_mode='1by1', hidden=32, homo_boost_thrd=0.5, homo_loss_weight=100, inner=1, inv_weight=1, lr=0.01, model='GCN', no_cuda=False, num_hidden=128, num_proj_hidden=128, prune_thr=0.2, seed=10, select_thrh=0.8, selection_method='none', target_class=0, target_loss_weight=1, tau=0.1, temperature=0.5, test_model='GCN', thrd=0.5, train_lr=0.01, trigger_size=3, trojan_epochs=400, use_vs_number=True, vs_number=10, vs_ratio=0, weight_decay=0.0005)


In [18]:
# from torch_geometric.utils import to_dense_adj, dense_to_sparse
# from torch_geometric.data import Data, InMemoryDataset

# import os

# import numpy as np
# from scipy.linalg import expm

# import torch
# from torch_geometric.data import Data, InMemoryDataset
# from torch_geometric.datasets import Planetoid, Amazon, Coauthor

# # from seeds import development_seed
# DATA_PATH = './data/'

# def get_dataset(name: str, use_lcc: bool = True) -> InMemoryDataset:
#     path = os.path.join(DATA_PATH, name)
#     if name in ['Cora', 'Citeseer', 'Pubmed']:
#         dataset = Planetoid(path, name)
#     elif name in ['Computers', 'Photo']:
#         dataset = Amazon(path, name)
#     elif name == 'CoauthorCS':
#         dataset = Coauthor(path, 'CS')
#     else:
#         raise Exception('Unknown dataset.')

#     if use_lcc:
#         lcc = get_largest_connected_component(dataset)

#         x_new = dataset.data.x[lcc]
#         y_new = dataset.data.y[lcc]

#         row, col = dataset.data.edge_index.numpy()
#         edges = [[i, j] for i, j in zip(row, col) if i in lcc and j in lcc]
#         edges = remap_edges(edges, get_node_mapper(lcc))
        
#         data = Data(
#             x=x_new,
#             edge_index=torch.LongTensor(edges),
#             y=y_new,
#             train_mask=torch.zeros(y_new.size()[0], dtype=torch.bool),
#             test_mask=torch.zeros(y_new.size()[0], dtype=torch.bool),
#             val_mask=torch.zeros(y_new.size()[0], dtype=torch.bool)
#         )
#         dataset.data = data

#     return dataset


# def get_component(dataset: InMemoryDataset, start: int = 0) -> set:
#     visited_nodes = set()
#     queued_nodes = set([start])
#     row, col = dataset.data.edge_index.numpy()
#     while queued_nodes:
#         current_node = queued_nodes.pop()
#         visited_nodes.update([current_node])
#         neighbors = col[np.where(row == current_node)[0]]
#         neighbors = [n for n in neighbors if n not in visited_nodes and n not in queued_nodes]
#         queued_nodes.update(neighbors)
#     return visited_nodes


# def get_largest_connected_component(dataset: InMemoryDataset) -> np.ndarray:
#     remaining_nodes = set(range(dataset.data.x.shape[0]))
#     comps = []
#     while remaining_nodes:
#         start = min(remaining_nodes)
#         comp = get_component(dataset, start)
#         comps.append(comp)
#         remaining_nodes = remaining_nodes.difference(comp)
#     return np.array(list(comps[np.argmax(list(map(len, comps)))]))


# def get_node_mapper(lcc: np.ndarray) -> dict:
#     mapper = {}
#     counter = 0
#     for node in lcc:
#         mapper[node] = counter
#         counter += 1
#     return mapper


# def remap_edges(edges: list, mapper: dict) -> list:
#     row = [e[0] for e in edges]
#     col = [e[1] for e in edges]
#     row = list(map(lambda x: mapper[x], row))
#     col = list(map(lambda x: mapper[x], col))
#     return [row, col]


# def get_adj_matrix(data) -> np.ndarray:
#     num_nodes = data.x.shape[0]
#     adj_matrix = np.zeros(shape=(num_nodes, num_nodes))
#     for i, j in zip(data.edge_index[0], data.edge_index[1]):
#         adj_matrix[i, j] = 1.
#     return adj_matrix


# def get_ppr_matrix(
#         adj_matrix: np.ndarray,
#         alpha: float = 0.1) -> np.ndarray:
#     num_nodes = adj_matrix.shape[0]
#     A_tilde = adj_matrix + np.eye(num_nodes)
#     D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
#     H = D_tilde @ A_tilde @ D_tilde
#     return alpha * np.linalg.inv(np.eye(num_nodes) - (1 - alpha) * H)


# def get_heat_matrix(
#         adj_matrix: np.ndarray,
#         t: float = 5.0) -> np.ndarray:
#     num_nodes = adj_matrix.shape[0]
#     A_tilde = adj_matrix + np.eye(num_nodes)
#     D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
#     H = D_tilde @ A_tilde @ D_tilde
#     return expm(-t * (np.eye(num_nodes) - H))


# def get_top_k_matrix(A: np.ndarray, k: int = 128) -> np.ndarray:
#     num_nodes = A.shape[0]
#     row_idx = np.arange(num_nodes)
#     A[A.argsort(axis=0)[:num_nodes - k], row_idx] = 0.
#     norm = A.sum(axis=0)
#     norm[norm <= 0] = 1 # avoid dividing by zero
#     return A/norm


# def get_clipped_matrix(A: np.ndarray, eps: float = 0.01) -> np.ndarray:
#     num_nodes = A.shape[0]
#     A[A < eps] = 0.
#     norm = A.sum(axis=0)
#     norm[norm <= 0] = 1 # avoid dividing by zero
#     return A/norm


# def set_train_val_test_split(
#         seed: int,
#         data: Data,
#         num_development: int = 1500,
#         num_per_class: int = 20) -> Data:
#     rnd_state = np.random.RandomState(development_seed)
#     num_nodes = data.y.shape[0]
#     development_idx = rnd_state.choice(num_nodes, num_development, replace=False)
#     test_idx = [i for i in np.arange(num_nodes) if i not in development_idx]

#     train_idx = []
#     rnd_state = np.random.RandomState(seed)
#     for c in range(data.y.max() + 1):
#         class_idx = development_idx[np.where(data.y[development_idx].cpu() == c)[0]]
#         train_idx.extend(rnd_state.choice(class_idx, num_per_class, replace=False))

#     val_idx = [i for i in development_idx if i not in train_idx]

#     def get_mask(idx):
#         mask = torch.zeros(num_nodes, dtype=torch.bool)
#         mask[idx] = 1
#         return mask

#     data.train_mask = get_mask(train_idx)
#     data.val_mask = get_mask(val_idx)
#     data.test_mask = get_mask(test_idx)

#     return data

# class PPRDataset(InMemoryDataset):
#     """
#     Dataset preprocessed with GDC using PPR diffusion.
#     Note that this implementations is not scalable
#     since we directly invert the adjacency matrix.
#     """
#     def __init__(self,noisy_data,
#                  name: str = 'Cora',
#                  use_lcc: bool = True,
#                  alpha: float = 0.1,
#                  k: int = 16,
#                  eps: float = None):
#         self.name = name
#         self.use_lcc = use_lcc
#         self.alpha = alpha
#         self.k = k
#         self.eps = eps
#         self.noisy_data = noisy_data

#         super(PPRDataset, self).__init__(DATA_PATH)
#         self.data, self.slices = torch.load(self.processed_paths[0])

#     @property
#     def raw_file_names(self) -> list:
#         return []

#     @property
#     def processed_file_names(self) -> list:
#         return [str(self) + '.pt']

#     def download(self):
#         pass

#     def process(self):
#         # base = get_dataset(name=self.name, use_lcc=self.use_lcc)
#         # generate adjacency matrix from sparse representation
#         adj_matrix = get_adj_matrix(self.noisy_data)
#         # obtain exact PPR matrix
#         ppr_matrix = get_ppr_matrix(adj_matrix,
#                                         alpha=self.alpha)

#         if self.k:
#             print(f'Selecting top {self.k} edges per node.')
#             ppr_matrix = get_top_k_matrix(ppr_matrix, k=self.k)
#         elif self.eps:
#             print(f'Selecting edges with weight greater than {self.eps}.')
#             ppr_matrix = get_clipped_matrix(ppr_matrix, eps=self.eps)
#         else:
#             raise ValueError

#         # create PyG Data object
#         edges_i = []
#         edges_j = []
#         edge_attr = []
#         for i, row in enumerate(ppr_matrix):
#             for j in np.where(row > 0)[0]:
#                 edges_i.append(i)
#                 edges_j.append(j)
#                 edge_attr.append(ppr_matrix[i, j])
#         edge_index = [edges_i, edges_j]

#         data = Data(
#             x=self.noisy_data.x,
#             edge_index=torch.LongTensor(edge_index),
#             edge_attr=torch.FloatTensor(edge_attr),
#             y=self.noisy_data.y,
#             train_mask=torch.zeros(self.noisy_data.train_mask.size()[0], dtype=torch.bool),
#             test_mask=torch.zeros(self.noisy_data.test_mask.size()[0], dtype=torch.bool),
#             val_mask=torch.zeros(self.noisy_data.val_mask.size()[0], dtype=torch.bool)
#         )

#         data, slices = self.collate([data])
#         torch.save((data, slices), self.processed_paths[0])

#     def __str__(self) -> str:
#         return f'{self.name}_ppr_alpha={self.alpha}_k={self.k}_eps={self.eps}_lcc={self.use_lcc}'


In [19]:
# from torch_geometric.utils import to_dense_adj, dense_to_sparse
# from torch_geometric.data import Data, InMemoryDataset

# import os

# import numpy as np
# from scipy.linalg import expm

# import torch
# from torch_geometric.data import Data, InMemoryDataset
# from torch_geometric.datasets import Planetoid, Amazon, Coauthor

# # from seeds import development_seed
# DATA_PATH = './data/'

# def get_dataset(name: str, use_lcc: bool = True) -> InMemoryDataset:
#     path = os.path.join(DATA_PATH, name)
#     if name in ['Cora', 'Citeseer', 'Pubmed']:
#         dataset = Planetoid(path, name)
#     elif name in ['Computers', 'Photo']:
#         dataset = Amazon(path, name)
#     elif name == 'CoauthorCS':
#         dataset = Coauthor(path, 'CS')
#     else:
#         raise Exception('Unknown dataset.')

#     if use_lcc:
#         lcc = get_largest_connected_component(dataset)

#         x_new = dataset.data.x[lcc]
#         y_new = dataset.data.y[lcc]

#         row, col = dataset.data.edge_index.numpy()
#         edges = [[i, j] for i, j in zip(row, col) if i in lcc and j in lcc]
#         edges = remap_edges(edges, get_node_mapper(lcc))
        
#         data = Data(
#             x=x_new,
#             edge_index=torch.LongTensor(edges),
#             y=y_new,
#             train_mask=torch.zeros(y_new.size()[0], dtype=torch.bool),
#             test_mask=torch.zeros(y_new.size()[0], dtype=torch.bool),
#             val_mask=torch.zeros(y_new.size()[0], dtype=torch.bool)
#         )
#         dataset.data = data

#     return dataset


# def get_component(dataset: InMemoryDataset, start: int = 0) -> set:
#     visited_nodes = set()
#     queued_nodes = set([start])
#     row, col = dataset.data.edge_index.numpy()
#     while queued_nodes:
#         current_node = queued_nodes.pop()
#         visited_nodes.update([current_node])
#         neighbors = col[np.where(row == current_node)[0]]
#         neighbors = [n for n in neighbors if n not in visited_nodes and n not in queued_nodes]
#         queued_nodes.update(neighbors)
#     return visited_nodes


# def get_largest_connected_component(dataset: InMemoryDataset) -> np.ndarray:
#     remaining_nodes = set(range(dataset.data.x.shape[0]))
#     comps = []
#     while remaining_nodes:
#         start = min(remaining_nodes)
#         comp = get_component(dataset, start)
#         comps.append(comp)
#         remaining_nodes = remaining_nodes.difference(comp)
#     return np.array(list(comps[np.argmax(list(map(len, comps)))]))


# def get_node_mapper(lcc: np.ndarray) -> dict:
#     mapper = {}
#     counter = 0
#     for node in lcc:
#         mapper[node] = counter
#         counter += 1
#     return mapper


# def remap_edges(edges: list, mapper: dict) -> list:
#     row = [e[0] for e in edges]
#     col = [e[1] for e in edges]
#     row = list(map(lambda x: mapper[x], row))
#     col = list(map(lambda x: mapper[x], col))
#     return [row, col]


# def get_adj_matrix(data) -> np.ndarray:
#     num_nodes = data.x.shape[0]
#     adj_matrix = np.zeros(shape=(num_nodes, num_nodes))
#     for i, j in zip(data.edge_index[0], data.edge_index[1]):
#         adj_matrix[i, j] = 1.
#     return adj_matrix


# def get_ppr_matrix(
#         adj_matrix: np.ndarray,
#         alpha: float = 0.1) -> np.ndarray:
#     num_nodes = adj_matrix.shape[0]
#     A_tilde = adj_matrix + np.eye(num_nodes)
#     D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
#     H = D_tilde @ A_tilde @ D_tilde
#     return alpha * np.linalg.inv(np.eye(num_nodes) - (1 - alpha) * H)


# def get_heat_matrix(
#         adj_matrix: np.ndarray,
#         t: float = 5.0) -> np.ndarray:
#     num_nodes = adj_matrix.shape[0]
#     A_tilde = adj_matrix + np.eye(num_nodes)
#     D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
#     H = D_tilde @ A_tilde @ D_tilde
#     return expm(-t * (np.eye(num_nodes) - H))


# def get_top_k_matrix(A: np.ndarray, k: int = 128) -> np.ndarray:
#     num_nodes = A.shape[0]
#     row_idx = np.arange(num_nodes)
#     A[A.argsort(axis=0)[:num_nodes - k], row_idx] = 0.
#     norm = A.sum(axis=0)
#     norm[norm <= 0] = 1 # avoid dividing by zero
#     return A/norm


# def get_clipped_matrix(A: np.ndarray, eps: float = 0.01) -> np.ndarray:
#     num_nodes = A.shape[0]
#     A[A < eps] = 0.
#     norm = A.sum(axis=0)
#     norm[norm <= 0] = 1 # avoid dividing by zero
#     return A/norm


# def set_train_val_test_split(
#         seed: int,
#         data: Data,
#         num_development: int = 1500,
#         num_per_class: int = 20) -> Data:
#     rnd_state = np.random.RandomState(development_seed)
#     num_nodes = data.y.shape[0]
#     development_idx = rnd_state.choice(num_nodes, num_development, replace=False)
#     test_idx = [i for i in np.arange(num_nodes) if i not in development_idx]

#     train_idx = []
#     rnd_state = np.random.RandomState(seed)
#     for c in range(data.y.max() + 1):
#         class_idx = development_idx[np.where(data.y[development_idx].cpu() == c)[0]]
#         train_idx.extend(rnd_state.choice(class_idx, num_per_class, replace=False))

#     val_idx = [i for i in development_idx if i not in train_idx]

#     def get_mask(idx):
#         mask = torch.zeros(num_nodes, dtype=torch.bool)
#         mask[idx] = 1
#         return mask

#     data.train_mask = get_mask(train_idx)
#     data.val_mask = get_mask(val_idx)
#     data.test_mask = get_mask(test_idx)

#     return data

# class PPRDataset(InMemoryDataset):
#     """
#     Dataset preprocessed with GDC using PPR diffusion.
#     Note that this implementations is not scalable
#     since we directly invert the adjacency matrix.
#     """
#     def __init__(self,noisy_data,
#                  name: str = 'Cora',
#                  use_lcc: bool = True,
#                  alpha: float = 0.1,
#                  k: int = 16,
#                  eps: float = None):
#         self.name = name
#         self.use_lcc = use_lcc
#         self.alpha = alpha
#         self.k = k
#         self.eps = eps
#         self.noisy_data = noisy_data

#         super(PPRDataset, self).__init__(DATA_PATH)
#         self.data, self.slices = torch.load(self.processed_paths[0])

#     @property
#     def raw_file_names(self) -> list:
#         return []

#     @property
#     def processed_file_names(self) -> list:
#         return [str(self) + '.pt']

#     def download(self):
#         pass

#     def process(self):
#         # base = get_dataset(name=self.name, use_lcc=self.use_lcc)
#         # generate adjacency matrix from sparse representation
#         adj_matrix = get_adj_matrix(self.noisy_data)
#         # obtain exact PPR matrix
#         ppr_matrix = get_ppr_matrix(adj_matrix,
#                                         alpha=self.alpha)

#         if self.k:
#             print(f'Selecting top {self.k} edges per node.')
#             ppr_matrix = get_top_k_matrix(ppr_matrix, k=self.k)
#         elif self.eps:
#             print(f'Selecting edges with weight greater than {self.eps}.')
#             ppr_matrix = get_clipped_matrix(ppr_matrix, eps=self.eps)
#         else:
#             raise ValueError

#         # create PyG Data object
#         edges_i = []
#         edges_j = []
#         edge_attr = []
#         for i, row in enumerate(ppr_matrix):
#             for j in np.where(row > 0)[0]:
#                 edges_i.append(i)
#                 edges_j.append(j)
#                 edge_attr.append(ppr_matrix[i, j])
#         edge_index = [edges_i, edges_j]

#         data = Data(
#             x=self.noisy_data.x,
#             edge_index=torch.LongTensor(edge_index),
#             edge_attr=torch.FloatTensor(edge_attr),
#             y=self.noisy_data.y,
#             train_mask=torch.zeros(self.noisy_data.train_mask.size()[0], dtype=torch.bool),
#             test_mask=torch.zeros(self.noisy_data.test_mask.size()[0], dtype=torch.bool),
#             val_mask=torch.zeros(self.noisy_data.val_mask.size()[0], dtype=torch.bool)
#         )

#         data, slices = self.collate([data])
#         torch.save((data, slices), self.processed_paths[0])

#     def __str__(self) -> str:
#         return f'{self.name}_ppr_alpha={self.alpha}_k={self.k}_eps={self.eps}_lcc={self.use_lcc}'


# Unify Contrastive GNN

## Backdoor

#### GNN+CL

In [20]:
for i in range(5):
    ''' Contrastive learning to backdoor in Contrastive learning'''
    from models.GTA import Backdoor
    import heuristic_selection as hs
    from models.GCN import GCN
    from models.construct import model_construct
    import copy 
    from model import UnifyModel, Encoder
    from models.construct import model_construct
    from models.GCN_CL import GCN_Encoder
    data = data.to(device)
    # learning_rate = 0.0002
    # weight_decay = config['weight_decay']
    num_class = int(data.y.max()+1)
    # args.cl_lr = 0.002
    # args.weight_decay = 0.0005

    size = args.vs_number #int((len(data.test_mask)-data.test_mask.sum())*args.vs_ratio)
    print("#Attach Nodes:{}".format(size))
    from models.construct import model_construct
    result_asr = []
    result_acc = []
    data = data.to(device)

    seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
    rs = np.random.RandomState(args.seed)
    seeds = rs.randint(1000,size=1)
    # seeds = [args.seed]
    for seed in seeds:
        np.random.seed(seed)
        # torch.manual_seed(seed)
        # torch.cuda.manual_seed(seed)
        args.seed = seed
        if(args.selection_method == 'none'):
            idx_attach = hs.obtain_attach_nodes(args,unlabeled_idx,size)
        elif(args.selection_method == 'cluster'):
            idx_attach = hs.cluster_distance_selection(args,data,idx_train,idx_val,idx_clean_test,unlabeled_idx,train_edge_index,size,device)
            idx_attach = torch.LongTensor(idx_attach).to(device)
        elif(args.selection_method == 'cluster_degree'):
            idx_attach = hs.cluster_degree_selection(args,data,idx_train,idx_val,idx_clean_test,unlabeled_idx,train_edge_index,size,device)
            idx_attach = torch.LongTensor(idx_attach).to(device)
        # train trigger generator 
        model = Backdoor(args,device)
        model.fit(data.x, train_edge_index, None, data.y, idx_train,idx_attach, unlabeled_idx)
        poison_x, poison_edge_index, poison_edge_weights, poison_labels = model.get_poisoned()

        from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated
        if(args.defense_mode == 'prune'):
            poison_edge_index,poison_edge_weights = prune_unrelated_edge(args,poison_edge_index,poison_edge_weights,poison_x,device,large_graph=False)
            bkd_tn_nodes = torch.cat([idx_train,idx_attach]).to(device)
        elif(args.defense_mode == 'isolate'):
            poison_edge_index,poison_edge_weights,rel_nodes = prune_unrelated_edge_isolated(args,poison_edge_index,poison_edge_weights,poison_x,device,large_graph=False)
            bkd_tn_nodes = torch.cat([idx_train,idx_attach]).tolist()
            bkd_tn_nodes = torch.LongTensor(list(set(bkd_tn_nodes) - set(rel_nodes))).to(device)
        else:
            bkd_tn_nodes = torch.cat([idx_train,idx_attach]).to(device)

        # bkd_tn_nodes = torch.cat([idx_train,idx_attach]).to(device)
        print("precent of left attach nodes: {:.3f}"\
            .format(len(set(bkd_tn_nodes.tolist()) & set(idx_attach.tolist()))/len(idx_attach)))
        #%%

        # model = GCN(nfeat=data.x.shape[1],\
        #             nhid=args.hidden,\
        #             nclass= int(data.y.max()+1),\
        #             dropout=args.dropout,\
        #             lr=args.train_lr,\
        #             weight_decay=args.weight_decay,\
        #             device=device)
        test_model = GCN_Encoder(args, data.x.shape[1], args.num_hidden, num_class, unlabeled_idx,dropout=0.5, lr=0.01, weight_decay=args.cl_weight_decay, tau=args.tau, layer=2,device=device,use_ln=False,layer_norm_first=False)
        test_model.fit(poison_x, poison_edge_index,poison_edge_weights,poison_labels,bkd_tn_nodes,idx_val=idx_val,train_iters=args.cl_num_epochs,seen_node_idx=seen_node_idx,verbose=True)
        # # test_model = model_construct(args,args.test_model,data,device).to(device) 
        # encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
        #                         base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
        # # test_model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=None).to(device)
        # test_model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
        # # test_model = UnifyModel(args, encoder, num_hidden, num_proj_hidden, num_class, tau, lr=learning_rate, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
        # test_model.fit(args, poison_x, poison_edge_index,poison_edge_weights,poison_labels,bkd_tn_nodes,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=seen_node_idx)
        test_embds = test_model(poison_x,poison_edge_index,poison_edge_weights)
        output = test_model.clf_head(test_embds)
        train_attach_rate = (output.argmax(dim=1)[idx_attach]==args.target_class).float().mean()
        print("target class rate on Vs: {:.4f}".format(train_attach_rate))
        torch.cuda.empty_cache()
        # test_model = GCN(nfeat=cont_poison_x.shape[1],\
        #             nhid=args.hidden,\
        #             nclass= int(data.y.max()+1),\
        #             dropout=args.dropout,\
        #             lr=args.train_lr,\
        #             weight_decay=args.weight_decay,\
        #             device=device).to(device) 
        # test_model.fit(cont_poison_x, poison_edge_index, poison_edge_weights, poison_labels, bkd_tn_nodes, idx_val,train_iters=args.epochs,verbose=False)

        # output = test_model(cont_poison_x,poison_edge_index,poison_edge_weights)
        # train_attach_rate = (output.argmax(dim=1)[idx_attach]==args.target_class).float().mean()
        # print("target class rate on Vs: {:.4f}".format(train_attach_rate))
        # torch.cuda.empty_cache()
        #%%
        induct_edge_index = torch.cat([poison_edge_index,mask_edge_index],dim=1)
        induct_edge_weights = torch.cat([poison_edge_weights,torch.ones([mask_edge_index.shape[1]],dtype=torch.float,device=device)])
        clean_acc = test_model.test(poison_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
        # test_model = test_model.cpu()

        print("accuracy on clean test nodes: {:.4f}".format(clean_acc))


        if(args.evaluate_mode == '1by1'):
            from torch_geometric.utils  import k_hop_subgraph
            overall_induct_edge_index, overall_induct_edge_weights = induct_edge_index.clone(),induct_edge_weights.clone()
            asr = 0
            flip_asr = 0
            flip_idx_atk = idx_atk[(data.y[idx_atk] != args.target_class).nonzero().flatten()]
            for i, idx in enumerate(idx_atk):
                idx=int(idx)
                sub_induct_nodeset, sub_induct_edge_index, sub_mapping, sub_edge_mask  = k_hop_subgraph(node_idx = [idx], num_hops = 2, edge_index = overall_induct_edge_index, relabel_nodes=True) # sub_mapping means the index of [idx] in sub)nodeset
                ori_node_idx = sub_induct_nodeset[sub_mapping]
                relabeled_node_idx = sub_mapping
                sub_induct_edge_weights = torch.ones([sub_induct_edge_index.shape[1]]).to(device)
                # inject trigger on attack test nodes (idx_atk)'''
                with torch.no_grad():
                    induct_x, induct_edge_index,induct_edge_weights = model.inject_trigger(relabeled_node_idx,poison_x[sub_induct_nodeset],sub_induct_edge_index,sub_induct_edge_weights,device)
                    induct_x, induct_edge_index,induct_edge_weights = induct_x.clone().detach(), induct_edge_index.clone().detach(),induct_edge_weights.clone().detach()
                    if(args.defense_mode == 'prune' or args.defense_mode == 'isolate'):
                        induct_edge_index,induct_edge_weights = prune_unrelated_edge(args,induct_edge_index,induct_edge_weights,induct_x,device,False)
        
                    test_embeds = test_model(induct_x, induct_edge_index,induct_edge_weights).to(device)
                    output = test_model.clf_head(test_embeds)
                    # cont_induct_x = contrastive_model(induct_x, induct_edge_index,induct_edge_weights).detach().to(device)
                    # # do pruning in test datas'''
                    # if(args.defense_mode == 'prune' or args.defense_mode == 'isolate'):
                    #     induct_edge_index,induct_edge_weights = prune_unrelated_edge(args,induct_edge_index,induct_edge_weights,induct_x,device,large_graph=False)
                    # attack evaluation

                    # output = test_model(cont_induct_x,induct_edge_index,induct_edge_weights)
                    train_attach_rate = (output.argmax(dim=1)[relabeled_node_idx]==args.target_class).float().mean()
                    # print("Node {}: {}, Origin Label: {}".format(i, idx, data.y[idx]))
                    # print("ASR: {:.4f}".format(train_attach_rate))
                    asr += train_attach_rate
                    if(data.y[idx] != args.target_class):
                        flip_asr += train_attach_rate
            asr = asr/(idx_atk.shape[0])
            flip_asr = flip_asr/(flip_idx_atk.shape[0])
            print("Overall ASR: {:.4f}".format(asr))
            print("Flip ASR: {:.4f}/{} nodes".format(flip_asr,flip_idx_atk.shape[0]))
        elif(args.evaluate_mode == 'overall'):
            # %% inject trigger on attack test nodes (idx_atk)'''
            induct_x, induct_edge_index,induct_edge_weights = model.inject_trigger(idx_atk,poison_x,induct_edge_index,induct_edge_weights,device)
            induct_x, induct_edge_index,induct_edge_weights = induct_x.clone().detach(), induct_edge_index.clone().detach(),induct_edge_weights.clone().detach()
            # do pruning in test datas'''
            if(args.defense_mode == 'prune' or args.defense_mode == 'isolate'):
                induct_edge_index,induct_edge_weights = prune_unrelated_edge(args,induct_edge_index,induct_edge_weights,induct_x,device)
            # attack evaluation

            test_embeds = test_model(induct_x, induct_edge_index,induct_edge_weights).to(device)
            output = test_model.clf_head(test_embeds)
            # test_model = test_model.to(device)
            # output = test_model(induct_x,induct_edge_index,induct_edge_weights)
            train_attach_rate = (output.argmax(dim=1)[idx_atk]==args.target_class).float().mean()
            print("ASR: {:.4f}".format(train_attach_rate))
            flip_idx_atk = idx_atk[(data.y[idx_atk] != args.target_class).nonzero().flatten()]
            flip_asr = (output.argmax(dim=1)[flip_idx_atk]==args.target_class).float().mean()
            print("Flip ASR: {:.4f}/{} nodes".format(flip_asr,flip_idx_atk.shape[0]))
            ca = test_model.test(induct_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
            print("CA: {:.4f}".format(ca))

        result_asr.append(float(asr))
        result_acc.append(float(clean_acc))

    print('The final ASR:{:.5f}, {:.5f}, Accuracy:{:.5f}, {:.5f}'\
                .format(np.average(result_asr),np.std(result_asr),np.average(result_acc),np.std(result_acc)))

#Attach Nodes:10
precent of left attach nodes: 1.000
=== training gcn model ===
Epoch 0, training loss: 26.898313522338867 = 1.953494906425476 + 3 * 7.664082050323486
Epoch 0, val loss: 1.939407467842102
Epoch 0, val acc: 0.2
Epoch 10, training loss: 26.39559555053711 = 1.9167629480361938 + 3 * 7.52122163772583
Epoch 10, val loss: 1.9424872398376465
Epoch 10, val acc: 0.124
Epoch 20, training loss: 25.13083267211914 = 1.8945739269256592 + 3 * 7.112472057342529
Epoch 20, val loss: 1.9418469667434692
Epoch 20, val acc: 0.124
Epoch 30, training loss: 24.502307891845703 = 1.8459302186965942 + 3 * 6.938636302947998
Epoch 30, val loss: 1.9187893867492676
Epoch 30, val acc: 0.11800000000000001
Epoch 40, training loss: 23.716419219970703 = 1.7346704006195068 + 3 * 6.748831272125244
Epoch 40, val loss: 1.859037160873413
Epoch 40, val acc: 0.152
Epoch 50, training loss: 22.844816207885742 = 1.4259687662124634 + 3 * 6.649346351623535
Epoch 50, val loss: 1.7625421285629272
Epoch 50, val acc: 0.384

In [9]:
import torch.nn.functional as F
T = 3
pred_prob = F.softmax(output[unlabeled_idx])
pred_prob.max(dim=1).indices[(pred_prob.max(dim=1).values>0).nonzero().flatten()]
# mean_pred_prob = torch.mean(F.softmax(output),dim=0)
# target_pred_prob = torch.pow(mean_pred_prob,1./T)
# target_pred_prob /= torch.mean(target_pred_prob)
# target_pred_prob

/tmp/ipykernel_40375/2119401665.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_prob = F.softmax(output[unlabeled_idx])
/opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [17,0,0], thread: [96,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [17,0,0], thread: [97,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [17,0,0], thread: [98,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [17,0,0], thr

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [8]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
from torch_geometric.explain import Explainer, GNNExplainer
from torch_geometric.nn import GCNConv

explainer = Explainer(
    model=test_model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)



for i in range(len(idx_attach)):
    node_index = idx_attach[i].item()
    explanation_cln = explainer(data.x, train_edge_index, index=node_index)
    print(f'Generated explanations in {explanation_cln.available_explanations}')

    path = './figures/cln_feature_importance_{}.png'.format(node_index)
    explanation_cln.visualize_feature_importance(path, top_k=10)
    print(f"Feature importance plot has been saved to '{path}'")

    path = './figures/cln_subgraph_{}.pdf'.format(node_index)
    explanation_cln.visualize_graph(path)
    print(f"Subgraph visualization plot has been saved to '{path}'")
for i in range(len(idx_attach)):
    node_index = idx_attach[i].item()
    explanation_bkd = explainer(poison_x, poison_edge_index, index=node_index)
    print(f'Generated explanations in {explanation_bkd.available_explanations}')

    path = './figures/bkd_feature_importance_{}.png'.format(node_index)
    explanation_bkd.visualize_feature_importance(path, top_k=10)
    print(f"Feature importance plot has been saved to '{path}'")

    path = './figures/bkd_subgraph_{}.pdf'.format(node_index)
    explanation_bkd.visualize_graph(path)
    print(f"Subgraph visualization plot has been saved to '{path}'")


Generated explanations in ['edge_mask', 'node_mask']
Feature importance plot has been saved to './figures/cln_feature_importance_1467.png'
Subgraph visualization plot has been saved to './figures/cln_subgraph_1467.pdf'
Generated explanations in ['edge_mask', 'node_mask']
Feature importance plot has been saved to './figures/cln_feature_importance_446.png'
Subgraph visualization plot has been saved to './figures/cln_subgraph_446.pdf'
Generated explanations in ['edge_mask', 'node_mask']
Feature importance plot has been saved to './figures/cln_feature_importance_1332.png'
Subgraph visualization plot has been saved to './figures/cln_subgraph_1332.pdf'
Generated explanations in ['edge_mask', 'node_mask']
Feature importance plot has been saved to './figures/cln_feature_importance_606.png'
Subgraph visualization plot has been saved to './figures/cln_subgraph_606.pdf'
Generated explanations in ['edge_mask', 'node_mask']
Feature importance plot has been saved to './figures/cln_feature_importance

In [14]:
print(explanation_bkd.edge_mask.nonzero())
print(explanation_bkd.edge_index[:,4447],explanation_bkd.edge_index[:,4457])

tensor([[4447],
        [4457]], device='cuda:3')
tensor([1564, 2735], device='cuda:3') tensor([2735, 1564], device='cuda:3')


In [41]:
explainer = Explainer(
    model=test_model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)

# node_index = idx_attach[0].item()
explanation_cln = explainer(data.x, train_edge_index, index=1332)
explanation_bkd = explainer(poison_x, poison_edge_index, index=1332)
# print(f'Generated explanations in {explanation.available_explanations}')

In [24]:
edge_mask_index = explanation.edge_mask.nonzero().flatten()
print(edge_mask_index)
print(explanation.edge_mask[edge_mask_index])
print(explanation.edge_index[:,edge_mask_index])

tensor([ 653, 1106, 1128, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143,
        1144, 1145, 1146, 1272, 1273, 1309, 1632, 1633, 1735, 2107, 2108, 2258,
        2259, 2273, 2276, 2277, 2609, 2611, 2868, 3368, 3369, 3842],
       device='cuda:3')
tensor([0.8975, 0.3870, 0.2195, 0.1435, 0.1484, 0.1446, 0.1448, 0.1502, 0.1387,
        0.1463, 0.1435, 0.1470, 0.1430, 0.8873, 0.1480, 0.6948, 0.1476, 0.2151,
        0.1445, 0.8789, 0.3935, 0.6912, 0.1440, 0.3895, 0.1461, 0.5392, 0.1432,
        0.1433, 0.2793, 0.1439, 0.2056, 0.9169, 0.8846, 0.5285],
       device='cuda:3')
tensor([[ 249,  424,  433,  436,  436,  436,  436,  436,  436,  436,  436,  436,
          436,  436,  436,  483,  483,  498,  621,  621,  668,  816,  816,  878,
          878,  885,  885,  885, 1039, 1039, 1131, 1332, 1332, 1494],
        [ 621,  436,  436,  424,  433,  483,  498,  668,  816,  878,  885, 1039,
         1131, 1332, 1494,  436,  816,  436,  249, 1332,  436,  436,  483,  436,
          885,  436,  87

In [39]:
a = np.array(list(set(explanation.edge_index[0,edge_mask_index].cpu().tolist())))
print(a)
print(poison_labels[a])

[ 483  424 1131  621  878 1039  816  433  498  436  885 1332 1494  249
  668]
tensor([5, 5, 3, 5, 5, 5, 5, 3, 4, 3, 5, 0, 5, 5, 5], device='cuda:3')


In [42]:
edge_mask_index = explanation_bkd.edge_mask.nonzero().flatten()
print(edge_mask_index)
print(explanation_bkd.edge_mask[edge_mask_index])
print(explanation_bkd.edge_index[:,edge_mask_index])
# explanation.edge_mask[edge_mask_index[0]]

tensor([ 653, 1106, 1128, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143,
        1144, 1145, 1146, 1272, 1273, 1309, 1632, 1633, 1735, 2107, 2108, 2258,
        2259, 2273, 2276, 2277, 2609, 2611, 2868, 3368, 3369, 3842, 4440, 4450],
       device='cuda:3')
tensor([0.3871, 0.1555, 0.2587, 0.1460, 0.1461, 0.1468, 0.1463, 0.1437, 0.1456,
        0.1464, 0.1495, 0.1438, 0.1454, 0.9041, 0.1492, 0.1828, 0.1456, 0.4570,
        0.1451, 0.8974, 0.1843, 0.2488, 0.1445, 0.5494, 0.1452, 0.1949, 0.1413,
        0.1451, 0.5756, 0.1435, 0.1741, 0.8342, 0.9003, 0.1696, 0.9068, 0.8778],
       device='cuda:3')
tensor([[ 249,  424,  433,  436,  436,  436,  436,  436,  436,  436,  436,  436,
          436,  436,  436,  483,  483,  498,  621,  621,  668,  816,  816,  878,
          878,  885,  885,  885, 1039, 1039, 1131, 1332, 1332, 1494, 1332, 2714],
        [ 621,  436,  436,  424,  433,  483,  498,  668,  816,  878,  885, 1039,
         1131, 1332, 1494,  436,  816,  436,  249, 1332,  436,  4

In [9]:
import torch.nn.functional as F
from torch_geometric.utils  import k_hop_subgraph
test_embds = test_model(poison_x,poison_edge_index,poison_edge_weights)
output = F.softmax(test_model.clf_head(test_embds))
# print(output[idx_attach])
# print(poison_labels[idx_attach])
# print(data.y[idx_attach])
unlabeled_idx
up = (output.argmax(dim=1)[unlabeled_idx]==data.y[unlabeled_idx]).float().mean()
print(up)
print((output.max(dim=1).values).cpu().detach().numpy()[::-1][:15])
print((output.max(dim=1).values<0.3).nonzero())
# print(output.argmax(dim=1).flip(dims=[0]))
print(output.values())

for idx in idx_attach:
    print("+=====")
    sub_induct_nodeset, sub_induct_edge_index, sub_mapping, sub_edge_mask = k_hop_subgraph(node_idx = [idx], num_hops = 2, edge_index = poison_edge_index, relabel_nodes=False) # sub_mapping means the index of [idx] in sub)nodeset
    # print(sub_induct_nodeset, sub_induct_edge_index, sub_mapping, sub_edge_mask)
    # print(sub_induct_nodeset[:-1:])
    up = (output.argmax(dim=1)[sub_induct_nodeset[:-1:]]==data.y[sub_induct_nodeset[:-1:]]).float().mean()
    print(up)
    up = output.max(dim=1).values[sub_induct_nodeset[:-1:]]
    print(up)
    print(data.y[sub_induct_nodeset[:-1:]])

/tmp/ipykernel_36324/938229985.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(test_model.clf_head(test_embds))


tensor(0.5963, device='cuda:2')
[0.37481055 0.37481055 1.         0.37481055 0.37481055 1.
 0.37481055 0.37481055 1.         0.37481055 0.37481055 0.9925921
 0.37481055 0.37481055 1.        ]
tensor([[  61],
        [  76],
        [  90],
        [ 140],
        [ 155],
        [ 156],
        [ 161],
        [ 162],
        [ 164],
        [ 170],
        [ 184],
        [ 191],
        [ 209],
        [ 211],
        [ 221],
        [ 244],
        [ 266],
        [ 268],
        [ 272],
        [ 273],
        [ 274],
        [ 277],
        [ 285],
        [ 289],
        [ 300],
        [ 304],
        [ 305],
        [ 314],
        [ 323],
        [ 324],
        [ 331],
        [ 340],
        [ 342],
        [ 355],
        [ 356],
        [ 359],
        [ 363],
        [ 367],
        [ 374],
        [ 376],
        [ 382],
        [ 389],
        [ 407],
        [ 413],
        [ 414],
        [ 424],
        [ 428],
        [ 432],
        [ 437],
        [ 446],
        

NotImplementedError: Could not run 'aten::values' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::values' is only available for these backends: [FuncTorchGradWrapper, Functionalize, MPS, IPU, UNKNOWN_TENSOR_TYPE_ID, QuantizedXPU, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseCPU, SparseCUDA, SparseHIP, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseXPU, UNKNOWN_TENSOR_TYPE_ID, SparseVE, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, NestedTensorCUDA, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE

In [ ]:
import torch.nn.functional as F
test_embds = test_model(poison_x,poison_edge_index,poison_edge_weights)
output = test_model.clf_head(test_embds)
# torch.var(output,unbiased = False)
sorted_pred =torch.argsort(output,dim=1,descending=False)
# print(output[0])
fst_prob = output[:,sorted_pred[:,0]][:,0].unsqueeze(dim=1)

# print(fst_prob)
# print(torch.concat([fst_prob,fst_prob],dim=1))

pred1 = torch.argmax(output,dim=1)
print(pred1)
print(F.cross_entropy(output,pred1))
# sec_prob = output[:,sorted_pred[:,1]][:,0]
# loss = (fst_prob - sec_prob)[[1,2,3]].mean()
# print(output.shape)
# print(sorted_pred[:,0].shape)
# print(fst_prob.shape)
# print(output)
# print(output[0,1])
# print(fst_prob)
# print(fst_prob[:,0])
# loss.data = torch.tensor(0).to(device)
# output
# output = self.clf_head(z)
# fst_prob = torch.argsort(output,dim=0)[0]
# sec_prob = torch.argsort(output,dim=0)[1]

### GNN

In [ ]:
'''Backdoor attack to GNN classifier'''
from models.GTA import Backdoor
import heuristic_selection as hs
from models.GCN import GCN
from models.construct import model_construct
from construct_graph import *
# args.homo_loss_weight=config['homo_loss_weight']
# args.vs_number=config['vs_number']
# args.trojan_epochs = config['trojan_epochs']

data = data.to(device)
num_class = int(data.y.max()+1)
# args.seed = config['seed']

size = args.vs_number #int((len(data.test_mask)-data.test_mask.sum())*args.vs_ratio)
print("#Attach Nodes:{}".format(size))
from models.construct import model_construct
result_asr = []
result_acc = []
data = data.to(device)

noisy_data = construct_noisy_graph(data,perturb_ratio=0.1,mode='random_noise')
data = noisy_data.to(device)
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=3)
# seeds = [args.seed]
for seed in seeds:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    args.seed = seed
    if(args.selection_method == 'none'):
        idx_attach = hs.obtain_attach_nodes(args,unlabeled_idx,size)
    elif(args.selection_method == 'cluster'):
        idx_attach = hs.cluster_distance_selection(args,data,idx_train,idx_val,idx_clean_test,unlabeled_idx,train_edge_index,size,device)
        idx_attach = torch.LongTensor(idx_attach).to(device)
    elif(args.selection_method == 'cluster_degree'):
        idx_attach = hs.cluster_degree_selection(args,data,idx_train,idx_val,idx_clean_test,unlabeled_idx,train_edge_index,size,device)
        idx_attach = torch.LongTensor(idx_attach).to(device)
    # train trigger generator 
    model = Backdoor(args,device)
    model.fit(data.x, train_edge_index, None, data.y, idx_train,idx_attach, unlabeled_idx)
    poison_x, poison_edge_index, poison_edge_weights, poison_labels = model.get_poisoned()
    bkd_tn_nodes = torch.cat([idx_train,idx_attach]).to(device)
    # # learn contrastive node representation
    # encoder = Encoder(dataset.num_features, num_hidden, activation,
    #                     base_model=base_model, k=num_layers).to(device)
    # contrastive_model = Model(encoder, num_hidden, num_proj_hidden, tau).to(device)
    # optimizer = torch.optim.Adam(
    #     contrastive_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # start = t()
    # prev = start
    # for epoch in range(1, num_epochs + 1):
    #     # loss = train_(model, data.x, data.edge_index)
    #     loss = train_1(contrastive_model, poison_x, poison_edge_index, poison_edge_weights)

    #     now = t()
    #     if(epoch%10 == 0):
    #         print(f'(T) | Epoch={epoch:03d}, loss={loss:.4f}, '
    #                 f'this epoch {now - prev:.4f}, total {now - start:.4f}')
    #     prev = now

    # contrastive_model.eval()
    # cont_poison_x = contrastive_model(poison_x, poison_edge_index, poison_edge_weights).detach().to(device)

    print("precent of left attach nodes: {:.3f}"\
        .format(len(set(bkd_tn_nodes.tolist()) & set(idx_attach.tolist()))/len(idx_attach)))
    #%%

    # model = GCN(nfeat=data.x.shape[1],\
    #             nhid=args.hidden,\
    #             nclass= int(data.y.max()+1),\
    #             dropout=args.dropout,\
    #             lr=args.train_lr,\
    #             weight_decay=args.weight_decay,\
    #             device=device)
    test_model = model_construct(args,args.test_model,data,device).to(device) 
    # test_model = GCN(nfeat=cont_poison_x.shape[1],\
    #             nhid=args.hidden,\
    #             nclass= int(data.y.max()+1),\
    #             dropout=args.dropout,\
    #             lr=args.train_lr,\
    #             weight_decay=args.weight_decay,\
    #             device=device).to(device) 
    test_model.fit(poison_x, poison_edge_index, poison_edge_weights, poison_labels, bkd_tn_nodes, idx_val,train_iters=args.epochs,verbose=False)
    test_model.eval()
    output = test_model(poison_x,poison_edge_index,poison_edge_weights)
    train_attach_rate = (output.argmax(dim=1)[idx_attach]==args.target_class).float().mean()
    print("target class rate on Vs: {:.4f}".format(train_attach_rate))
    torch.cuda.empty_cache()
    #%%
    induct_edge_index = torch.cat([poison_edge_index,mask_edge_index],dim=1)
    induct_edge_weights = torch.cat([poison_edge_weights,torch.ones([mask_edge_index.shape[1]],dtype=torch.float,device=device)])
    clean_acc = test_model.test(poison_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
    # test_model = test_model.cpu()

    print("accuracy on clean test nodes: {:.4f}".format(clean_acc))


    if(args.evaluate_mode == '1by1'):
        from torch_geometric.utils  import k_hop_subgraph
        overall_induct_edge_index, overall_induct_edge_weights = induct_edge_index.clone(),induct_edge_weights.clone()
        asr = 0
        flip_asr = 0
        flip_idx_atk = idx_atk[(data.y[idx_atk] != args.target_class).nonzero().flatten()]
        for i, idx in enumerate(idx_atk):
            idx=int(idx)
            sub_induct_nodeset, sub_induct_edge_index, sub_mapping, sub_edge_mask  = k_hop_subgraph(node_idx = [idx], num_hops = 2, edge_index = overall_induct_edge_index, relabel_nodes=True) # sub_mapping means the index of [idx] in sub)nodeset
            ori_node_idx = sub_induct_nodeset[sub_mapping]
            relabeled_node_idx = sub_mapping
            sub_induct_edge_weights = torch.ones([sub_induct_edge_index.shape[1]]).to(device)
            # inject trigger on attack test nodes (idx_atk)'''
            with torch.no_grad():
                induct_x, induct_edge_index,induct_edge_weights = model.inject_trigger(relabeled_node_idx,poison_x[sub_induct_nodeset],sub_induct_edge_index,sub_induct_edge_weights,device)
                induct_x, induct_edge_index,induct_edge_weights = induct_x.clone().detach(), induct_edge_index.clone().detach(),induct_edge_weights.clone().detach()
                # cont_induct_x = contrastive_model(induct_x, induct_edge_index,induct_edge_weights).detach().to(device)
                # # do pruning in test datas'''
                # if(args.defense_mode == 'prune' or args.defense_mode == 'isolate'):
                #     induct_edge_index,induct_edge_weights = prune_unrelated_edge(args,induct_edge_index,induct_edge_weights,induct_x,device,large_graph=False)
                # attack evaluation

                output = test_model(induct_x,induct_edge_index,induct_edge_weights)
                train_attach_rate = (output.argmax(dim=1)[relabeled_node_idx]==args.target_class).float().mean()
                # print("Node {}: {}, Origin Label: {}".format(i, idx, data.y[idx]))
                # print("ASR: {:.4f}".format(train_attach_rate))
                asr += train_attach_rate
                if(data.y[idx] != args.target_class):
                    flip_asr += train_attach_rate
        asr = asr/(idx_atk.shape[0])
        flip_asr = flip_asr/(flip_idx_atk.shape[0])
        print("Overall ASR: {:.4f}".format(asr))
        print("Flip ASR: {:.4f}/{} nodes".format(flip_asr,flip_idx_atk.shape[0]))
    result_asr.append(float(asr))
    result_acc.append(float(clean_acc))
print('The final ASR:{:.5f}, {:.5f}, Accuracy:{:.5f}, {:.5f}'\
            .format(np.average(result_asr),np.std(result_asr),np.average(result_acc),np.std(result_acc)))

In [ ]:
test_model = model_construct(args,args.test_model,data,device).to(device) 
    # test_model = GCN(nfeat=cont_poison_x.shape[1],\
    #             nhid=args.hidden,\
    #             nclass= int(data.y.max()+1),\
    #             dropout=args.dropout,\
    #             lr=args.train_lr,\
    #             weight_decay=args.weight_decay,\
    #             device=device).to(device) 
test_model.fit(poison_x, poison_edge_index, poison_edge_weights, poison_labels, bkd_tn_nodes, idx_val,train_iters=args.epochs,verbose=False)

output = test_model(poison_x,poison_edge_index,poison_edge_weights)
train_attach_rate = (output.argmax(dim=1)[idx_attach]==args.target_class).float().mean()
print("target class rate on Vs: {:.4f}".format(train_attach_rate))
torch.cuda.empty_cache()
#%%
induct_edge_index = torch.cat([poison_edge_index,mask_edge_index],dim=1)
induct_edge_weights = torch.cat([poison_edge_weights,torch.ones([mask_edge_index.shape[1]],dtype=torch.float,device=device)])
clean_acc = test_model.test(poison_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
# test_model = test_model.cpu()

print("accuracy on clean test nodes: {:.4f}".format(clean_acc))

In [ ]:
import torch.nn.functional as F
test_embds = test_model(poison_x,poison_edge_index,poison_edge_weights)
# output = test_model.fc(test_embds)
# output = test_embds
output = test_model.clf_head(test_embds)
# output = F.log_softmax(test_model.clf_head(test_embds),dim=1)
train_attach_rate = (output.argmax(dim=1)[idx_attach]==args.target_class).float().mean()
print(train_attach_rate)
clean_acc = test_model.test(poison_x,induct_edge_index,induct_edge_weights,data.y,idx_clean_test)
print("accuracy on clean test nodes: {:.4f}".format(clean_acc))
test_embds = test_model(poison_x,induct_edge_index,induct_edge_weights)
poisoned_embds = test_embds[idx_attach]

# # cos = F.CosineSimilarity(dim=1, eps=1e-6)
# for idx in idx_attach:
#     target_class_idx = (data.y!=args.target_class).nonzero().flatten()
#     cos_sim = F.cosine_similarity(poisoned_embds[idx],test_embds[target_class_idx[:5]])
#     print(cos_sim)

for idx in idx_attach:
    # get neighbor nodes
    nbs_0 = (poison_edge_index[0]==idx).nonzero().flatten()
    idx_nbs = poison_edge_index[1,nbs_0]
    idx_labeled_nbs = idx_nbs[(idx_nbs<poison_labels.shape[0]).nonzero().flatten()]
    print(output[idx])
    # print("idx_nbs",idx_nbs,output[idx_nbs])
    # print("poison_labels",poison_labels[idx_labeled_nbs])
    # nbs_1 = (poison_edge_index[1]==idx).nonzero().flatten()
    # nbs = torch.cat([nbs_0,nbs_1],dim=0)
    # print(test_embds.shape)
    # print(idx,poison_edge_index[1,nbs_0])
    # print(idx,poison_edge_index[0,nbs_1])
    # print(poison_x[idx].repeat(4,1).shape)
    # print(nbs)
    # print((poison_edge_index[0]==idx).nonzero().flatten())
    # print((poison_edge_index[1]==idx).nonzero().flatten())
    # target_class_idx = (data.y!=args.target_class).nonzero().flatten()
    cos_sim = F.cosine_similarity(F.softmax(output[idx].repeat(len(idx_labeled_nbs),1)),F.softmax(output[idx_labeled_nbs]))
    print("Sim",cos_sim,poison_labels[idx_labeled_nbs])
    # print(idx_nbs)

In [ ]:
a = torch.tensor([[-2.1612e+00, -7.9176e+00, -2.4735e+01, -1.2284e-01, -1.3994e+01,
         -1.0212e+01, -2.0797e+01]])
b = torch.tensor([[   0.0000,  -43.7667, -108.6508,  -23.4506,  -69.0643,  -43.1416,
        -103.0275]])

c = torch.tensor([[  -1.1746,   -2.5948,   -6.4277,   -0.6488,   -3.3728,   -2.9037,
           -5.8016]])
d = torch.tensor([[  -1.2395,   -2.4253,   -5.1802,   -0.7167,   -2.7565,   -2.8887,
           -4.7231]])


# print(F.cosine_similarity(F.softmax(c),F.softmax(d)))
print(output.shape)